In [1]:
from FileSystem import *
import os
home_dir = os.getcwd()
print(home_dir)
import datetime as dt
from datetime import *
curtz = datetime.now().astimezone().tzinfo
tform = '%Y-%m-%d %H-%M-%S%z'
import re
import json

D:\Users\feelus\Repos\hash_sync


In [2]:
def print_tree(obj):
    s = myjson_dumps(nested_join(obj))
    # проверка на кол-во строк ...
    print(s)
def load_snapshot(path):
    tmp = strip_json_scomments(myjson_load(path))
    return nested_split(tmp['root']),nested_split(tmp['errors'])
def dump_snapshot(root,errors,path):
    myjson_dump({
        'errors':nested_join(errors),
        'root':nested_join(root)
    },path)
def find_date_file(prefix,postfix,ls):
    return [s[len(prefix):-len(postfix)] for s in ls if \
              s.startswith(prefix) and s.endswith(postfix)]

In [3]:
newtime_s = None
snapshot_json = None
oldtime_s = None
snapshot_bak = None
root = None
oldroot = None
errors = None
olderrors = None
patch = None
def last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot,global_log=None):
    emergency_dump = True
    global newtime_s
    global oldtime_s
    global root
    global oldroot
    global errors
    global olderrors
    global patch
    try:
        os.chdir(prefix_)
    except Exception as e:
        if global_log:
            with open(global_log,'a') as gl:
                gl.write(prefix+'\t'+'device unavailable')
            return
        else:
            raise e
    loclog = open('.files/log.txt','a')
    globlog = open(global_log,'a') if global_log else None
    def log(s):
        loclog.write(str(s))
        loclog.flush()
        if globlog:
            globlog.write(prefix+'\t'+str(s))
            globlog.flush()
        else:
            raise Exception(s)
    
    try:
        newtime = datetime.now(curtz)
        newtime_s = newtime.strftime(tform)
        snapshot_json = '.files/last_snapshot '+newtime_s+'.json'
        def find_date_file(prefix,postfix,ls):
            return [s[len(prefix):-len(postfix)] for s in ls if \
                      s.startswith(prefix) and s.endswith(postfix)]
        if '.files' in os.listdir('.'):
            ls = os.listdir('.files\\')
            oldtime_list = find_date_file('last_snapshot ','.bak',ls)
            assert len(oldtime_list)<=1
            if len(oldtime_list)==1:
                oldtime_s = oldtime_list[0]
                snapshot_bak = '.files/last_snapshot '+oldtime_s+'.bak'

                oldroot,olderrors = load_snapshot(snapshot_bak) # <----
                newtime_list = find_date_file('last_snapshot ','.json',ls)
                assert len(newtime_list)<=1
                if len(newtime_list)==1:
                    print('fast recovery')
                    # сканируем-пересчитываем на основе .json
                    # затем удаляем его и сохраняем новый
                    newtime_s = newtime_list[0]
                    snapshot_json = '.files/last_snapshot '+newtime_s+'.json'

                    root,errors = load_snapshot(snapshot_json) # <----
                    #root,errors = scan(prefix_,exclude_dirs)
                    #calc_hashes(root,errors,oldnewroot,prefix)

                    #os.remove(oldsnapshot_json)
                    #dump_snapshot(root,errors,snapshot_json) # ---->
                else:
                    print('recovery')
                    # сканируем-пересчитываем на основе .bak
                    # затем сохраняем новый
                    root,errors = scan(prefix_,exclude_dirs)
                    calc_hashes(root,errors,oldroot,prefix)

                    dump_snapshot(root,errors,snapshot_json) # ---->
            else:
                print('simple update')
                # сканируем-пересчитываем на основе json
                # затем переименовываем его в .bak и сохраняем новый
                oldtime_list = find_date_file('last_snapshot ','.json',ls)
                assert len(oldtime_list)==1
                oldtime_s = oldtime_list[0]
                snapshot_bak = '.files/last_snapshot '+oldtime_s+'.bak'

                oldroot,olderrors = load_snapshot('.files/last_snapshot '+oldtime_s+'.json') # <----
                root,errors = scan(prefix_,exclude_dirs)
                calc_hashes(root,errors,oldroot,prefix)

                os.rename('.files/last_snapshot '+oldtime_s+'.json',snapshot_bak)
                dump_snapshot(root,errors,snapshot_json) # ---->
        else:
            print('create all')
            # записываем пустое дерево в .bak (дата на минуту раньше текущей)
            # сканируем-пересчитываем на основе .bak
            # затем сохраняем новый
            os.mkdir('.files')
            oldtime_s = (newtime - timedelta(minutes=1)).strftime(tform)
            snapshot_bak = '.files/last_snapshot '+oldtime_s+'.bak'
            oldroot = {}
            root,errors = scan(prefix_,exclude_dirs)
            calc_hashes(root,errors,oldroot,prefix)

            dump_snapshot(oldroot,{},snapshot_bak) # ---->
            dump_snapshot(root,errors,snapshot_json)
    except BaseException as e:
        log(Exception('scan:',e))
        return

    try:
        oldroot_d = oldroot#{k:v for k,v in oldroot.items() if k!='__scan_errors__'}
        root_d = root#{k:v for k,v in root.items() if k!='__scan_errors__'}
        modified,old,new,strict_old,strict_new,touched = path_diff(oldroot_d,root_d)
        assert set(old)&set(strict_old) == set()
        assert set(new)&set(strict_new) == set()
        moved,old_dirs,new_dirs,old,new = hash_diff(old,new)
        assert set(old)&set(strict_old) == set()
        assert set(new)&set(strict_new) == set()
        e_modified,e_old,e_new,e_strict_old,e_strict_new,e_touched = path_diff(olderrors,errors)

        patch = {
            'errors':path_patch_compress(e_modified,e_old,e_new,e_strict_old,e_strict_new,e_touched),
            'root':hash_patch_compress(modified,moved,old_dirs,new_dirs,
                                  {**old,**strict_old},{**new,**strict_new},touched)
        }

        modified_d,moved_d,old_dirs_d,new_dirs_d,old_d,new_d,touched_d = \
            hash_patch_uncompress(patch['root'])
        newoldroot = hash_back_patch(root_d,modified_d,moved_d,old_dirs_d,new_dirs_d,
                              old_d,new_d,touched_d)
        assert oldroot_d==newoldroot

        e_modified_d,e_old_d,e_new_d,e_strict_old_d,e_strict_new_d,e_touched_d = \
            path_patch_uncompress(patch['errors'])
        newerrors = path_patch(olderrors,e_modified_d,
                {**e_old_d,**e_strict_old_d},{**e_new_d,**e_strict_new_d}, e_touched_d)
        assert errors==newerrors
        
    except BaseException as e:
        if emergency_dump:
            print('exception catched, writing "exception_dump.json"')
            with open('.files/exception_dump.json','w') as file:
                json.dump({'olderrors':olderrors,'errors':errors,'oldroot':oldroot,
                           'root':root},      file)
        log(Exception('diff:',e))
        return
        
    try:
        myjson_dump(patch,'.files/patch '+oldtime_s+' to '+newtime_s+'.json')
        if find_date_file('first_snapshot ','.json',ls):
            os.remove(snapshot_bak)
        else:
            os.rename(snapshot_bak,'.files/first_snapshot '+oldtime_s+'.json',)
    except BaseException as e:
        log(Exception('scan:',e))
        return
        
    loclog.close()
    if globlog: globlog.close()
    

# D

In [4]:
exclude_dirs = {r'D:\Users\feelus\YandexDisk',r'D:\$RECYCLE.BIN'}
prefix = 'D:'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

fast recovery
--- ITERATION ---
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2020-07-10 19-23-44+0300 to 2020-07-11 16-05-10+0300.json


# C

In [7]:
exclude_dirs = {r'C:\$RECYCLE.BIN'}
prefix = 'C:'
prefix_ = prefix+'\\'
def en_load_snapshot(path):
    with open(path,'r') as file:
        tmp = strip_json_scomments(json.load(file))
    return nested_split(tmp['root']),nested_split(tmp['errors'])
def en_dump_snapshot(root,errors,path):
    with open(path,'w') as file:
        json.dump({
            'errors':nested_join(errors),
            'root':nested_join(root)
        },file,indent='\t')

last_diff_dir(prefix,prefix_,exclude_dirs,en_load_snapshot,en_dump_snapshot)

simple update
141 GB scanned - completed
141 GB calculated, 1 GB calculated (99% cached)- completed
--- ITERATION ---
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2020-07-10 20-46-48+0300 to 2020-07-11 17-21-10+0300.json


# YD

In [8]:
exclude_dirs = {}
prefix = r'D:\Users\feelus\YandexDisk'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

simple update
207 GB scanned - completed
207 GB calculated, 0 GB calculated (99% cached)- completed
start writing
writed .files/last_snapshot 2020-07-11 17-32-35+0300.json
--- ITERATION ---
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2020-07-10 20-59-29+0300 to 2020-07-11 17-32-35+0300.json


In [7]:
tree_stat(root)

(222633560483, 12161, 133130, 0)

# H

In [8]:
exclude_dirs = {r'H:\$RECYCLE.BIN'}
prefix = 'H:'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

simple update
343 GB scanned - completed
343 GB calculated, 0 GB calculated (99% cached)- completed
start writing
writed .files/last_snapshot 2020-07-10 21-00-46+0300.json
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2020-07-10 00-29-28+0300 to 2020-07-10 21-00-46+0300.json


# I

In [9]:
exclude_dirs = {r'I:\$RECYCLE.BIN'}
prefix = 'I:'
prefix_ = prefix+'\\'
last_diff_dir(prefix,prefix_,exclude_dirs,load_snapshot,dump_snapshot)

simple update
330 GB scanned - completed
330 GB calculated, 0 GB calculated (99% cached)- completed
start writing
writed .files/last_snapshot 2020-07-10 21-00-48+0300.json
--- ITERATION ---
--- FINAL ITERATION ---
start writing
writed .files/patch 2020-07-10 00-29-30+0300 to 2020-07-10 21-00-48+0300.json


In [ ]:
import os
os.system("shutdown /s /t 1");

In [ ]:
raise BaseException()

# Debug

In [ ]:
myjson_dumps({chr(0):0})

In [ ]:
'\x00'

In [ ]:
import json

with open(r'.files/last_snapshot 2019-11-18 14-25-23+3000.bak','r') as file:
    tmp = tree_load(json.load(file))
    
del tmp['root']['Windows']['System32']['LogFiles']['HTTPERR']['httperr1.log']

with open(r'.files/last_snapshot 2019-11-18 14-25-23+3000.bak','w') as file:
    json.dump(tree_dump(tmp),file,indent='\t')


In [ ]:
from copy import *

len(old)
old_d = {}#deepcopy(old)
keys = [k for k in old.keys()]
for k in keys:#[2554:]:
    old_d[k]=deepcopy(old[k])
print(len(old_d))

len(new)
new_d = {}#deepcopy(old)
keys = [k for k in new.keys()]
for k in keys:#[3164:-1]:
    new_d[k]=deepcopy(new[k])
print(len(new_d))

#print(myjson_dumps(path_patch_dump({},old_d,new_d,{},{})))

r = hash_diff(old_d,new_d)


In [ ]:
old_root = myjson_load(r'D:\.files\111.json')
root = myjson_load(r'D:\.files\222.json')

#root = scan('D:\\',{r'D:\Users\feelus\YandexDisk',r'D:\$RECYCLE.BIN'})
#calc_hashes(root,old_root,'D:')

In [ ]:
oldroot_d = oldroot#{k:v for k,v in oldroot.items() if k!='__scan_errors__'}
root_d = root#{k:v for k,v in root.items() if k!='__scan_errors__'}
modified,old,new,strict_old,strict_new,touched = path_diff(oldroot_d,root_d)
assert set(old)&set(strict_old) == set()
assert set(new)&set(strict_new) == set()
moved,old_dirs,new_dirs,old,new = hash_diff(old,new)
assert set(old)&set(strict_old) == set()
assert set(new)&set(strict_new) == set()

newoldroot = hash_back_patch(root_d,modified,moved,old_dirs,new_dirs,
                      {**old,**strict_old},{**new,**strict_new},touched)
#first_diff(root_d,new_root_d)
assert oldroot_d==newoldroot



In [ ]:
if 0:
    old = {
        ('a','b'):[1,1,'1']
    }
    new = {
        ('b','c'):[2,2,'2']
    }
if 0:
    old = {
        ('a','b'):[1,1,'1']
    }
    new = {
        ('b','c'):[2,2,'1']
    }
if 0:
    old = {
        ('a','b'):[1,1,'1'],
        ('a','c'):[1,1,'1'],
    }
    new = {
        ('b','c'):[2,2,'1']
    }
if 1:
    old = {
        ('a','b'):[1,1,'1'],
        ('a','c'):[1,1,'1'],
    }
    new = {
        ('b','c'):[2,2,'1'],
        ('b','d'):[2,2,'1'],
    }
r = hash_diff(old,new)

print('moved',r[0])
print('old',r[1])
print('new',r[2])
print('old1',r[3])
print('new1',r[4])


In [ ]:
oldroot['.files']['patch 2019-11-24 02-40-04+0300 to 2019-11-25 00-37-04+0300.json']

# D debug

In [ ]:
exclude_dirs = {r'D:\Users\feelus\YandexDisk',r'D:\$RECYCLE.BIN'}
prefix = 'D:'
prefix_ = prefix+'\\'
os.chdir(prefix_)

In [ ]:
oldroot,olderrors = load_snapshot('.files/2019-12-08 01-50-34+0300 snapshot.json');
root,errors = load_snapshot('.files/2019-12-09 00-18-35+0300 init_snapshot.json');

In [ ]:
root['Users']['feelus']['AppData']['Roaming']['Typora']['themes']['old-themes'].keys()

In [ ]:
kkk = {
    'github.css', 
    
}
oldroot_d = {k:v for k,v in oldroot['Users']['feelus']['AppData']['Roaming']['Typora']['themes']['old-themes'].items() if k in kkk}
root_d = {k:v for k,v in root['Users']['feelus']['AppData']['Roaming']['Typora']['themes']['old-themes'].items() if k in kkk}
oldroot_d = oldroot
root_d = root
#print(oldroot_d)
#print(root_d)

In [ ]:
modified,old,new,strict_old,strict_new,touched = path_diff(oldroot_d,root_d)
assert set(old)&set(strict_old) == set()
assert set(new)&set(strict_new) == set()
moved,old_dirs,new_dirs,old,new = hash_diff(old,new)
assert set(old)&set(strict_old) == set()
assert set(new)&set(strict_new) == set()
#print(modified)
#print(touched)
#print(old)
#print(new)

In [ ]:
newoldroot = hash_back_patch(root_d,modified,moved,old_dirs,new_dirs,
                      {**old,**strict_old},{**new,**strict_new},touched)
#first_diff(root_d,new_root_d)
assert oldroot_d==newoldroot

e_modified,e_old,e_new,e_strict_old,e_strict_new,e_touched = path_diff(olderrors,errors)
newerrors = path_patch(olderrors,e_modified,
                       {**e_old,**e_strict_old},{**e_new,**e_strict_new}, e_touched)
assert errors==newerrors
print('OK')

In [ ]:
patch_name = '.files/2019-12-08 01-50-34+0300 to 2019-12-09 00-18-35+0300 patch.json'
myjson_dump({
    'errors':path_patch_dump(e_modified,e_old,e_new,e_strict_old,e_strict_new,e_touched),
    'root':hash_patch_dump(modified,moved,old_dirs,new_dirs,
                          {**old,**strict_old},{**new,**strict_new},touched)
},patch_name)


In [ ]:
doc = myjson_load('D://.files/patch 2020-07-05 00-25-06+0300 to 2020-07-08 00-15-44+0300.json')
hp = doc['root']
tmp = pathlist2moved(moved2pathlist(hp['moved']))
moved = hp['moved']
hp['moved'] = moved2pathlist(hp['moved'])

In [ ]:
x1=set()#moved
x2=set()#tmp
for i in range(len(tmp)):
    x1.add(tuple(moved[i]))
    x2.add(tuple(tmp[i]))
x1==x2

In [ ]:
tmp = action2tree(hp)
tmp2 = tree2action(tmp)
tmp2==hp

In [ ]:
tmp3 = nested_join(tmp,'',lambda x: not x.startswith('/'),lambda x:x)

In [ ]:
tmp==nested_split(tmp3,'/',False,lambda x: not x.startswith('/'),lambda x:x)

In [ ]:
test = {'/f':{'new':"1594120574.2637675 32500327884 d9d08e1a0837cc42112fb62da4a1211c"}}
statistics(test,300_000_000)
test

In [ ]:
os.chdir('D://.files')
statistics(tmp3,300_000_000)
myjson_dump(tmp3,'test.json')

In [ ]:
nested_split(nested_join({
    'x':{
        'y':[1,2,'3'],
        'z':[1,2,'3'],
    }
}))

In [ ]:
'/a/b/c'.split('/')

In [ ]:
y=None
def foo(a=1,b=2,x=y):
    print(y(73))
def yy(x):
    return x
y=yy
foo()

In [4]:
os.chdir(r'C:\\.files\\')
ls = os.listdir('.')
ols = []#find_date_file('patch ','.json',ls)
for s in ols:
    os.rename('patch '+s+'.json',s+' patch.json',)

In [5]:
ls = os.listdir('.')
ols = find_date_file('',' patch.json',ls)
def en_load_snapshot(path):
    with open(path,'r') as file:
        tmp = strip_json_scomments(json.load(file))
    return nested_split(tmp['root']),nested_split(tmp['errors'])
def en_dump_snapshot(root,errors,path):
    with open(path,'w') as file:
        json.dump({
            'errors':nested_join(errors),
            'root':nested_join(root)
        },file,indent='\t')

for s in []:#ols:
    with open(s+' patch.json','r') as file:
        patch = json.load(file)

    hp = patch['errors']
    #hp['moved'] = moved2pathlist(hp['moved'])
    hp = action2tree(hp)
    hp = nested_join(hp,'',lambda x: not x.startswith('/'),lambda x:x)
    statistics(hp,300_000_000)
    patch['errors']=hp

    hp = patch['root']
    hp['moved'] = moved2pathlist(hp['moved'])
    hp = action2tree(hp)
    hp = nested_join(hp,'',lambda x: not x.startswith('/'),lambda x:x)
    statistics(hp,300_000_000)
    patch['root']=hp

    print('patch '+s+'.json')
    myjson_dump(patch,'patch '+s+'.json')


In [6]:
def check_list(path='.',start = None):
    ls = os.listdir(path)
    ols = find_date_file('patch ','.json',ls)
    r = {}
    l = {}
    for s in ols:
        f,t = s.split(' to ')
        r[f]=t
        l[t]=f
    if not start:
        start = next(iter(r))
    lst = [start]
    while lst[0] in l:
        cur = lst[0]
        lst.insert(0,l[cur])
        del r[l[cur]]
        del l[cur]
    while lst[-1] in r:
        cur = lst[-1]
        lst.append(r[cur])
        del l[r[cur]]
        del r[cur]
    #assert len(l)==len(r)
    if len(r)>0:
        print('осталось',len(r),'переходов')
    return lst


In [7]:
def patch_chain(lst,frm,to,in_snapshot):
    if type(frm)!=int:
        frm=lst.index(frm)
    assert frm>=0 and frm<len(lst)
    if type(to)!=int:
        to=lst.index(to)
    assert to>=0 and to<len(lst)
    errors = in_snapshot['errors']
    root = in_snapshot['root']
    print(frm,to)
    if frm<to: # to right
        for i in range(frm,to):
            path = 'patch '+lst[i]+' to '+lst[i+1]+'.json'
            print(path)
            patch = myjson_load(path)
            
            e_modified_d,e_old_d,e_new_d,e_strict_old_d,e_strict_new_d,e_touched_d = \
                path_patch_uncompress(patch['errors'])
            errors = path_patch(errors,e_modified_d,
                    {**e_old_d,**e_strict_old_d},{**e_new_d,**e_strict_new_d}, e_touched_d)
            
            modified_d,moved_d,old_dirs_d,new_dirs_d,old_d,new_d,touched_d = \
                hash_patch_uncompress(patch['root'])
            root = hash_patch(root,modified_d,moved_d,old_dirs_d,new_dirs_d,
                                  old_d,new_d,touched_d)

    elif frm>to: # to left
        for i in range(frm,to,-1):
            path = 'patch '+lst[i-1]+' to '+lst[i]+'.json'
            print('back',path)
            patch = myjson_load(path)
            
            e_modified_d,e_old_d,e_new_d,e_strict_old_d,e_strict_new_d,e_touched_d = \
                path_patch_uncompress(patch['errors'])
            errors = path_back_patch(errors,e_modified_d,
                    {**e_old_d,**e_strict_old_d},{**e_new_d,**e_strict_new_d}, e_touched_d)
            
            modified_d,moved_d,old_dirs_d,new_dirs_d,old_d,new_d,touched_d = \
                hash_patch_uncompress(patch['root'])
            root = hash_back_patch(root,modified_d,moved_d,old_dirs_d,new_dirs_d,
                                  old_d,new_d,touched_d)
    return {'errors':errors,'root':root}

In [8]:
os.chdir(r'C:\\.files\\')
lst = check_list()
lst

['2019-11-18 14-25-23+3000',
 '2019-11-21 11-37-35+0300',
 '2019-11-21 12-24-38+0300',
 '2019-11-22 01-08-21+0300',
 '2019-11-23 00-09-47+0300',
 '2019-11-24 02-43-50+0300',
 '2019-11-25 00-40-57+0300',
 '2019-11-25 23-33-25+0300',
 '2019-11-30 01-52-54+0300',
 '2019-12-03 01-55-50+0300',
 '2019-12-06 01-32-24+0300',
 '2019-12-06 16-07-50+0300',
 '2019-12-07 01-23-41+0300',
 '2019-12-08 01-54-08+0300',
 '2019-12-09 00-32-42+0300',
 '2019-12-10 00-54-49+0300',
 '2019-12-11 00-31-21+0300',
 '2019-12-12 00-14-30+0300',
 '2019-12-12 23-17-39+0300',
 '2019-12-14 00-48-15+0300',
 '2019-12-15 01-21-23+0300',
 '2019-12-16 00-50-13+0300',
 '2019-12-17 00-43-01+0300',
 '2019-12-18 01-34-24+0300',
 '2019-12-20 03-19-32+0300',
 '2019-12-21 03-25-42+0300',
 '2019-12-23 02-12-07+0300',
 '2019-12-23 23-34-10+0300',
 '2019-12-26 00-49-59+0300',
 '2019-12-28 00-48-04+0300',
 '2019-12-30 01-29-03+0300',
 '2019-12-31 01-35-48+0300',
 '2020-01-02 02-44-55+0300',
 '2020-01-05 02-45-25+0300',
 '2020-01-06 0

In [12]:
def en_load_snapshot(path):
    with open(path,'r') as file:
        tmp = strip_json_scomments(json.load(file))
    return nested_split(tmp['root']),nested_split(tmp['errors'])
root,errs = en_load_snapshot('last_snapshot 2020-07-10 00-15-35+0300.bak1')
snap79 = {'errors':errs,'root':root}

In [13]:
snap81_1 = patch_chain(lst,'2020-07-10 00-15-35+0300','2020-07-11 17-21-10+0300',snap79)

79 81
patch 2020-07-10 00-15-35+0300 to 2020-07-10 20-46-48+0300.json
patch 2020-07-10 20-46-48+0300 to 2020-07-11 17-21-10+0300.json
('.files', 'last_snapshot 2020-07-10 00-15-35+0300.json') ('.files', 'last_snapshot 2020-07-10 00-15-35+0300.bak1')


In [12]:
snap78 = patch_chain(lst,'2020-07-10 00-15-35+0300','2020-07-08 00-25-23+0300',snap79)

79 78
back patch 2020-07-08 00-25-23+0300 to 2020-07-10 00-15-35+0300.json


KeyError: 'Р’РѕСЃСЃС‚Р°РЅРѕРІР»РµРЅРёРµ СЃРµСЂРІРёСЃР° РѕР±РЅРѕРІР»РµРЅРёР№ РЇРЅРґРµРєСЃ.Р‘СЂР°СѓР·РµСЂР°.job'

In [14]:
def en_load_snapshot(path):
    with open(path,'r') as file:
        tmp = strip_json_scomments(json.load(file))
    return nested_split(tmp['root']),nested_split(tmp['errors'])
root,errs = en_load_snapshot('last_snapshot 2020-07-11 17-21-10+0300.json')
snap81 = {'errors':errs,'root':root}

In [16]:
snap80_2 = patch_chain(lst,'2020-07-11 17-21-10+0300','2020-07-10 20-46-48+0300',snap81)

81 80
back patch 2020-07-10 20-46-48+0300 to 2020-07-11 17-21-10+0300.json
('.files', 'last_snapshot 2020-07-10 00-15-35+0300.bak1') ('.files', 'last_snapshot 2020-07-10 00-15-35+0300.json')


In [18]:
snap79_1 = patch_chain(lst,'2020-07-11 17-21-10+0300','2020-07-10 00-15-35+0300',snap81)

81 79
back patch 2020-07-10 20-46-48+0300 to 2020-07-11 17-21-10+0300.json
('.files', 'last_snapshot 2020-07-10 00-15-35+0300.bak1') ('.files', 'last_snapshot 2020-07-10 00-15-35+0300.json')
back patch 2020-07-10 00-15-35+0300 to 2020-07-10 20-46-48+0300.json


In [17]:
snap81==snap81_1

True

In [19]:
snap79==snap79_1

True